# Notes

## Matrix Factorization

Matrix factorization is to factorize a matrix, i.e. to find out two (or more) matrices such that when you multiply them you will get back the original matrix.

### Intuition

The intuition behind using matrix factorization to solve this problem is that there should be some latent features that determine how a user rates an item. For example, two users would give high ratings to a certain movie if they both like the actors or actresses in the movie, or if the movie is an action movie, which is a genre preferred by both users.

Hence, if we can discover these latent features, we should be able to predict a rating with respect to a certain user and a certain item, because the features associated with the user should match with the features associated with the item.


### Math

Firstly, we have a set $U$ of users, and a set $D$ of items.<br/>Let $R$ of size $|U| \times |D|$ be the matrix that contains all the ratings that the users have assigned to the items.<br/>We assume that we would like to discover $K$ latent features.<br/>We must then find two matrics matrices P (of size $|U| \times |K|$) and Q (of size  $|D| \times |K|$) such that their product apprioximates $|R|$:

$$\mathbf{R} \approx \mathbf{P} \times \mathbf{Q}^T = \hat{\mathbf{R}}$$

Each row of $P$ would represent the strength of the associations between **a user and the features**.<br/>Each row of $Q$ would represent the strength of the associations between **an item and the features**.<br/>To get the prediction of a rating of an item $d_j$ by $u_i$, we can calculate the dot product of their vectors:

$$\hat{r}_{ij} = p_i^T q_j = \sum_{k=1}^k{p_{ik} q_{kj}}$$

Now, we have to find a way to obtain $P$ and $Q$. One way to approach this problem is the first intialize the two matrices with some values, calculate how different their product is to $R$, and then try to minimize this difference iteratively through **gradient descent**. The difference here, usually called the error between the estimated rating and the real rating, can be calculated by the following equation for each user-item pair:

$$e_{ij}^2 = (r_{ij} - \hat{r}_{ij})^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2$$

**Squared error** is considered here. To minimize the error, we have to know in which direction we have to modify the values of $p_{ik}$ and $q_{kj}$. We know this by getting the gradient at the current values by differentiating the above equation with respect to $p_{ik}$ and $q_{kj}$ separately:

$$\frac{\partial}{\partial p_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(q_{kj}) = -2 e_{ij} q_{kj}$$

$$\frac{\partial}{\partial q_{ik}}e_{ij}^2 = -2(r_{ij} - \hat{r}_{ij})(p_{ik}) = -2 e_{ij} p_{ik}$$

The update rules for the values will thus be:

$$p’_{ik} = p_{ik} + \alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + 2\alpha e_{ij} q_{kj}$$

$$q’_{kj} = q_{kj} + \alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + 2\alpha e_{ij} p_{ik}$$

$\mathbf{\alpha}$ is the learning rate, which should be small (say 0.0002) to avoid the event of oscillating around the minimum.

**Note:** If we find two matrices $\mathbf{P}$ and $\mathbf{Q}$ such that $\mathbf{P} \times \mathbf{Q}$ approximates $\mathbf{R}$, isn’t that our predictions of all the unseen ratings will be zeros? In fact, we are **not** really trying to come up with $\mathbf{P}$ and $\mathbf{Q}$ such that we can reproduce $\mathbf{R}$ exactly. Instead, **we will only try to minimise the errors of the observed user-item pairs**. In other words, if we let $T$ be a set of tuples, each of which is in the form of $(u_i, d_j, r_{ij})$, such that $T$ contains all the observed user-item pairs together with the associated ratings, we are only trying to minimise every $e_{ij}$ for $(u_i, d_j, r_{ij}) \in T$. (In other words, $T$ is our set of training data.) As for the rest of the unknowns, we will be able to determine their values once the associations between the users, items and features have been learnt.

Using the above update rules, we can then iteratively perform the operation until the error converges to its minimum. We can check the overall error as calculated using the following equation and determine when we should stop the process.

$$E = \sum_{(u_i, d_j, r_{ij}) \in T}{e_{ij}} = \sum_{(u_i,d_j,r_{ij}) \in T}{(r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2}$$

### Regularization

Regularization avoids overfitting. It can be done by adding a $\beta$ parameter to the square error:

$$e_{ij}^2 = (r_{ij} - \sum_{k=1}^K{p_{ik}q_{kj}})^2 + \frac{\beta}{2} \sum_{k=1}^K{(||P||^2 + ||Q||^2)}$$

The $\beta$ parameter controls the **magnitudes** of the user-feature and item-feature vectors such that $P$ and $Q$ would give predictions of $R$ that are not too large. In practice, $\beta$ is set to some values in the order of 0.02. The new update rules for this squared error can be obtained by a procedure similar to the one described above. The new update rules are as follows: 

$$p’_{ik} = p_{ik} + \alpha \frac{\partial}{\partial p_{ik}}e_{ij}^2 = p_{ik} + \alpha(2 e_{ij} q_{kj} - \beta p_{ik} )$$

$$q’_{kj} = q_{kj} + \alpha \frac{\partial}{\partial q_{kj}}e_{ij}^2 = q_{kj} + \alpha(2 e_{ij} p_{ik} - \beta q_{kj} )$$


### Biases

Adding biases will better model how a particular user rates items. E.g. A person who is a big movie fan may rate movies more strictly and thus give lower ratings. Biases can be added to the prediction rating as follows:

$$\hat{r}_{ij} = b + bu_i + bd_j + \sum_{k=1}^k{p_{ik} q_{kj}}$$

$b$ - global bias (mean of all ratings not including missing ratings)<br/>
$bu_i$ - bias for user $i$<br/>
$bd_j$ - bias for item $j$<br/>

We can derive the update rules for user and item biases as follows:

$$bu’_i = bu_i + \alpha \times (e_{ij} - \beta bu_i)$$

$$bd’_j = bd_j + \alpha \times (e_{ij} - \beta bd_j)$$


Source: http://www.albertauyeung.com/post/python-matrix-factorization/

## Simple Implementation

In [1]:
import numpy as np

class MF():

    def __init__(self, R, K, alpha, beta, iterations):
        """
        Perform matrix factorization to predict empty
        entries in a matrix.

        Arguments
        - R (ndarray)   : user-item rating matrix
        - K (int)       : number of latent dimensions
        - alpha (float) : learning rate
        - beta (float)  : regularization parameter
        """

        self.R = R
        self.num_users, self.num_items = R.shape
        self.K = K
        self.alpha = alpha
        self.beta = beta
        self.iterations = iterations

    def train(self):
        # Initialize user and item latent feature matrice
        self.P = np.random.normal(scale=1./self.K, size=(self.num_users, self.K))
        self.Q = np.random.normal(scale=1./self.K, size=(self.num_items, self.K))

        # Initialize the biases
        self.b_u = np.zeros(self.num_users)
        self.b_i = np.zeros(self.num_items)
        self.b = np.mean(self.R[np.where(self.R != 0)])

        # Create a list of training samples
        self.samples = [
            (i, j, self.R[i, j])
            for i in range(self.num_users)
            for j in range(self.num_items)
            if self.R[i, j] > 0
        ]

        # Perform stochastic gradient descent for number of iterations
        training_process = []
        for i in range(self.iterations):
            np.random.shuffle(self.samples)
            self.sgd()
            mse = self.mse()
            training_process.append((i, mse))
            if (i+1) % 10 == 0:
                print("Iteration: %d ; error = %.4f" % (i+1, mse))

        return training_process

    def mse(self):
        """
        A function to compute the total mean square error
        """
        xs, ys = self.R.nonzero()
        predicted = self.full_matrix()
        error = 0
        for x, y in zip(xs, ys):
            error += pow(self.R[x, y] - predicted[x, y], 2)
        return np.sqrt(error)

    def sgd(self):
        """
        Perform stochastic graident descent
        """
        for i, j, r in self.samples:
            # Compute prediction and error
            prediction = self.get_rating(i, j)
            e = (r - prediction)

            # Update biases
            self.b_u[i] += self.alpha * (e - self.beta * self.b_u[i])
            self.b_i[j] += self.alpha * (e - self.beta * self.b_i[j])

            # Update user and item latent feature matrices
            self.P[i, :] += self.alpha * (e * self.Q[j, :] - self.beta * self.P[i,:])
            self.Q[j, :] += self.alpha * (e * self.P[i, :] - self.beta * self.Q[j,:])

    def get_rating(self, i, j):
        """
        Get the predicted rating of user i and item j
        """
        prediction = self.b + self.b_u[i] + self.b_i[j] + self.P[i, :].dot(self.Q[j, :].T)
        return prediction

    def full_matrix(self):
        """
        Compute the full matrix using the resultant biases, P and Q
        """
        return self.b + self.b_u[:,np.newaxis] + self.b_i[np.newaxis:,] + self.P.dot(self.Q.T)

In [2]:
# Define User - Item rating matrix
R = np.array([
    [5, 3, 0, 1],
    [4, 0, 0, 1],
    [1, 1, 0, 5],
    [1, 0, 0, 4],
    [0, 1, 5, 4],
])

# Initialize the matrix factorization calss
mf = MF(R, K=2, alpha=0.1, beta=0.01, iterations=20)

In [3]:
# Train model
mf.train()

Iteration: 10 ; error = 0.2757
Iteration: 20 ; error = 0.0914


[(0, 5.614837535644302),
 (1, 4.512964958079523),
 (2, 2.9139616029573214),
 (3, 1.3017359773823518),
 (4, 0.6832182842853068),
 (5, 0.5116811778466939),
 (6, 0.4225289245999003),
 (7, 0.3624919353220784),
 (8, 0.3152072539488986),
 (9, 0.2756590562790787),
 (10, 0.2367585017063062),
 (11, 0.2067988690195686),
 (12, 0.184155713253405),
 (13, 0.16357454282110012),
 (14, 0.14499570475381018),
 (15, 0.13211346334986868),
 (16, 0.11842730501953525),
 (17, 0.10385746272525037),
 (18, 0.09725279369541372),
 (19, 0.0913767178449509)]

In [4]:
mf.mse()

0.0913767178449509

In [5]:
# For reference
print(R)

mf.get_rating(0, 2)

[[5 3 0 1]
 [4 0 0 1]
 [1 1 0 5]
 [1 0 0 4]
 [0 1 5 4]]


2.9703147403819377

In [6]:
mf.full_matrix()

array([[4.99440691, 2.98796957, 2.97031474, 1.01278321],
       [3.99195387, 2.10030051, 2.29014907, 1.0084241 ],
       [1.03955357, 0.96293669, 4.85106426, 4.98688867],
       [1.00529166, 0.95529937, 4.94761083, 3.9946126 ],
       [1.15179831, 1.06785112, 4.9918014 , 3.99396041]])